# set up 

In [2]:
!pip install msgpack
!pip install --upgrade pip

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 1.3MB 23.3MB/s ta 0:00:01
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [3]:
%%bash
CFFI_VERSION=$(pip list 2>/dev/null | grep cffi )
echo $CFFI_VERSION
if [[ "$CFFI_VERSION" == "cffi (1.10.0)" ]]
then 
   pip uninstall --yes cffi
fi
yum_log=$(sudo yum install -y libffi-devel openssl-devel)
pip_log=$(pip install --upgrade snowflake-connector-python)  
if [[ "$CFFI_VERSION" == "cffi (1.10.0)" ]]
then 
   echo "configuration has changed; restart notebook"
fi

cffi 1.11.5


https://nvidia.github.io/libnvidia-container/amzn1/x86_64/repodata/repomd.xml: [Errno -1] repomd.xml signature could not be verified for libnvidia-container
Trying other mirror.
https://nvidia.github.io/nvidia-container-runtime/amzn1/x86_64/repodata/repomd.xml: [Errno -1] repomd.xml signature could not be verified for nvidia-container-runtime
Trying other mirror.
https://nvidia.github.io/nvidia-docker/amzn1/x86_64/repodata/repomd.xml: [Errno -1] repomd.xml signature could not be verified for nvidia-docker
Trying other mirror.


In [4]:
import snowflake.connector
# Gets the version
ctx = snowflake.connector.connect(
user='XXX',
password='XXX',
account='XXX',
role='XXX'
)
cs = ctx.cursor()
try:
    cs.execute("SELECT current_version()")
    one = cs.fetchone()
    print(one[0])
    cs.execute("USE warehouse DS_XLARGE")

finally:
    cs.close()

3.6.4


# import packages

In [5]:
import pandas as pd
import numpy as np
import scipy as sp

from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering

# Clustering 

## data prep 

### read in data: diabetes clickers device IDs and associated features in long format

In [6]:
sql0='''
select * from  ds.diabetes1203.diab_seg_temp_new_id; 
'''


In [7]:
cur = ctx.cursor()
try:
    results0 = cur.execute(sql0).fetchall()
finally:
    cur.close()

In [8]:
features_long = pd.DataFrame(np.array(results0), columns = ['DEVICE_ID_HASHED', 'SEGMENT_ID_NEW'])

In [9]:
features_long.shape

(2853523, 2)

### reshape data from long to wide 

In [10]:
features_pivot=features_long.pivot_table(index=["DEVICE_ID_HASHED"], columns="SEGMENT_ID_NEW", aggfunc=lambda x: 1, fill_value=0)

In [11]:
features_pivot.shape

(60033, 359)

In [12]:
features_pivot.head()

SEGMENT_ID_NEW,1,10,100,101,1016,1017,1018,1019,102,1020,...,90,91,92,93,94,95,96,97,98,99
DEVICE_ID_HASHED,,,,,,,,,,,,,,,,,,,,,
00001971424878D0CC19699ABC274C1E847CB4C5,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
0000EDF2D3E8596891941567E396B915A23F78D0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00014A2530170777E36C476C3BF53D9D232F81DF,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
00036D75F6224F3EDDE72B5A210AFC039EA144D3,0,0,0,1,0,1,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
00041FE78147183DF84FF8BFD70448456CCD5599,0,1,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0


### Calculate pairwise distances with the dice dissimilarity.
http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html

In [13]:
m = metrics.pairwise.pairwise_distances(features_pivot, features_pivot, metric='dice') 


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


### remove missing values in dissimilarity matrix 
missing values in dissimilarity matric came from false false values in both entries. 
They create negative infinite values in dissimilarity matrix.
Though nothing is deleted in this dataset.

In [14]:
m=pd.DataFrame(m)
keep=m.index[pd.isnull(m).any(axis=0)==False]
m1=m.iloc[keep,keep]

In [15]:
m1.shape

(60033, 60033)

### Hierarchical Clustering 
Perform agglomerative clustering.
The affinity is precomputed (since the distances are precalculated).
'Aaverage' linkage is used. Other linkages could also be used except  'ward'.

In [16]:
agg = AgglomerativeClustering(n_clusters=700, affinity='precomputed',
                              linkage='average')

# Use the distance matrix directly.
cluster_agg = agg.fit_predict(m1)

### Compute Silhouette score 
resource on standards of evaluating silhouette score: https://www.stat.berkeley.edu/~spector/s133/Clus.html

In [17]:
metrics.silhouette_score(m1, cluster_agg, metric="precomputed")

0.7370919271556617

## Add Predicted Clusters to data in wide format 

In [18]:
features_pivot['Cluster_ID']=cluster_agg
features_pivot.head(3)

SEGMENT_ID_NEW,1,10,100,101,1016,1017,1018,1019,102,1020,...,91,92,93,94,95,96,97,98,99,Cluster_ID
DEVICE_ID_HASHED,,,,,,,,,,,,,,,,,,,,,
00001971424878D0CC19699ABC274C1E847CB4C5,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,179
0000EDF2D3E8596891941567E396B915A23F78D0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,39
00014A2530170777E36C476C3BF53D9D232F81DF,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,171


## format data to feed into Tableau 

## subset above table and create table with only deviec IDs and cluster IDs

In [19]:
device_cluster0=pd.DataFrame([features_pivot.index, cluster_agg]).T
device_cluster=device_cluster0.rename(index=str, columns={0: "DEVICE_ID_HASHED", 1: "Cluster_ID"})
device_cluster.head()

,DEVICE_ID_HASHED,Cluster_ID
0,00001971424878D0CC19699ABC274C1E847CB4C5,179
1,0000EDF2D3E8596891941567E396B915A23F78D0,39
2,00014A2530170777E36C476C3BF53D9D232F81DF,171
3,00036D75F6224F3EDDE72B5A210AFC039EA144D3,492
4,00041FE78147183DF84FF8BFD70448456CCD5599,598


### merge above table with device ID and feature data in long format
so that ID, features, cluster IDs are in long format 

In [20]:
device_cluster_long=device_cluster.merge(features_long, left_on='DEVICE_ID_HASHED',right_on='DEVICE_ID_HASHED')

### merge above table with feature dictionary 

In [22]:
cur = ctx.cursor()
try:
    ref = cur.execute('select * from ds.public.location_seg_taxonomies ').fetchall()
finally:
    cur.close()

In [23]:
ref = pd.DataFrame(np.array(ref), columns = ['CATEGORY','SEGMENT_NAME_NEW','SEGMENT_ID_NEW'])

In [24]:
device_cluster_long_ref=device_cluster_long.merge(ref,left_on='SEGMENT_ID_NEW', right_on='SEGMENT_ID_NEW')

In [31]:
device_cluster_long_ref.head()

,DEVICE_ID_HASHED,Cluster_ID,SEGMENT_ID_NEW,CATEGORY,SEGMENT_NAME_NEW
0,00001971424878D0CC19699ABC274C1E847CB4C5,179,290,Purchaser,Toothpaste/Toothbrushes
1,00041FE78147183DF84FF8BFD70448456CCD5599,598,290,Purchaser,Toothpaste/Toothbrushes
2,0006521B7ED4FA17F256D175837CDD6962574FDC,274,290,Purchaser,Toothpaste/Toothbrushes
3,000DCB16D79071E6A4C978B954AE0594C055F04A,83,290,Purchaser,Toothpaste/Toothbrushes
4,000F2FC01685ACE6BE109F3D898A0885ABC666FC,635,290,Purchaser,Toothpaste/Toothbrushes


## compute cluster size and merge it with above table 

In [38]:
unique_elements, counts_elements = np.unique(cluster_agg, return_counts=True)
cluster_size=pd.DataFrame([unique_elements, counts_elements]).T.rename(columns={0:'Cluster_ID',1:'cluster_size'})

final=device_cluster_long_ref.merge(cluster_size, left_on='Cluster_ID', right_on='Cluster_ID', how='left')

## format data for output

In [39]:
final_sorted=final.sort_values(by=['cluster_size'], ascending=False)

In [40]:
final_sorted.to_csv('resultHCdiab_sub_loc_All_features1214.csv')